In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pylab as pyp
from skimage import io, data

plt.rcParams.update({'figure.max_open_warning': 0})

def resize_image(image, padding):
    """
    This function lets you resize an image for working with some operations that require
    adding (or extend) the borders of the input image, specially for Convolution and
    Binarization methods like Niblack's and Sauvola's.
    Returns the image resized.
    """
    image_height, image_width = image.shape
    new_image_height, new_image_width = 2 * padding + image_height, 2 * padding + image_width
    new_image = np.zeros((new_image_height, new_image_width))
    
    # Copy the original image at the center of the new extended image.
    new_image[padding:new_image_height - padding, padding:new_image_width - padding] = image
    
    # Get all borders
    image_left_border = np.array([image[:,0]]).T
    image_right_border = np.array([image[:,-1]]).T
    image_top_border = image[0,:]
    image_bottom_border = image[-1,:]
    
    # Get all corners
    # LT : Left-Top    | RT : Right-Top
    # LB : Left-Bottom | RB : Right-Bottom
    image_lt_corner = np.array([image[0,0]])
    image_rt_corner = np.array([image[0,-1]])
    image_lb_corner = np.array([image[-1,0]])
    image_rb_corner = np.array([image[-1,-1]])
    
    # Copy the left border
    new_image[padding:image_height + padding,
              0:padding] = np.tile(image_left_border, (1, padding))
    
    # Copy the right border
    new_image[padding:image_height + padding,
              image_width + padding:new_image_width] = np.tile(image_right_border, (1, padding))
    
    # Copy the top border
    new_image[0:padding,
              padding:image_width + padding] = np.tile(image_top_border, (padding, 1))

    # Copy the bottom border
    new_image[image_height + padding:new_image_height,
              padding:image_width + padding] = np.tile(image_bottom_border, (padding, 1))
    
    # Copy the corners
    new_image[0:padding, 0:padding] = np.tile(image_lt_corner, (padding, padding))
    new_image[0:padding, image_width + padding:new_image_width] = np.tile(image_rt_corner, (padding, padding))
    new_image[image_height + padding:new_image_height, 0:padding] = np.tile(image_lb_corner, (padding, padding))
    new_image[image_height + padding:new_image_height, image_width + padding:new_image_width] = np.tile(image_rb_corner, (padding, padding))
    
    return new_image

def diffusion_coefficient(gradient_value, K, option = 1):
    if option == 1: return np.exp(-(np.abs(gradient_value)/K)**2)
    elif option == 2: return 1 / (1 + (np.abs(gradient_value)/K)**2)
    else: return None
    
def anisotropic_diffusion(image, steps = 100, K = 1, _lambda = 0.25, diff_coef = 1):
    new_image = np.copy(image)
    for t in np.arange(steps):
        extended_image = resize_image(new_image, padding=1)
        difference_north = extended_image[:-2, 1:-1] - extended_image[1:-1, 1:-1]
        difference_south = extended_image[2:, 1:-1]  - extended_image[1:-1, 1:-1] 
        difference_east  = extended_image[1:-1, 2:]  - extended_image[1:-1, 1:-1] 
        difference_west  = extended_image[1:-1, :-2] - extended_image[1:-1, 1:-1]
        new_image = new_image + _lambda * \
                    (diffusion_coefficient(difference_north, K, diff_coef) * difference_north + \
                     diffusion_coefficient(difference_south, K, diff_coef) * difference_south + \
                     diffusion_coefficient(difference_east, K, diff_coef) * difference_east + \
                     diffusion_coefficient(difference_west, K, diff_coef) * difference_west)
    return new_image

def load_images():
    image_set = []
    for i in np.arange(5):
        image_path = "original_images/{}.png".format(i)
        image_set.append(io.imread(image_path))
        image_path = "noisy_images/{}.png".format(i)
        image_set.append(io.imread(image_path))
    return image_set

def perform_anisotropic_diffusion(image, steps_list, K_list, lambda_list):
    print(">> Anisotropic Diffusion evaluation has started!")
    varying_steps_result_set = []
    varying_K_result_set = []
    varying_lambda_result_set = []

    # First, we perform Anisotropic Diffusion varying the steps.
    # Constant values are: K = 4 and lambda = 0.25
    for steps in steps_list:
        first_result  = anisotropic_diffusion(image, steps = steps, K = 4, _lambda = 0.25, diff_coef = 1)
        second_result = anisotropic_diffusion(image, steps = steps, K = 4, _lambda = 0.25, diff_coef = 2)
        varying_steps_result_set.append([first_result, second_result])

    # Second, we perform Anisotropic Diffusion varying the K-value.
    # Constant values are: steps = 100 and lambda = 0.25
    for K in K_list:
        first_result  = anisotropic_diffusion(image, steps = 100, K = K, _lambda = 0.25, diff_coef = 1)
        second_result = anisotropic_diffusion(image, steps = 100, K = K, _lambda = 0.25, diff_coef = 2)
        varying_K_result_set.append([first_result, second_result])

    # Third, we perform Anisotropic Diffusion varying the lambda.
    # Constant values are: steps = 100 and K = 4
    for _lambda in lambda_list:
        first_result  = anisotropic_diffusion(image, steps = 100, K = 4, _lambda = _lambda, diff_coef = 1)
        second_result = anisotropic_diffusion(image, steps = 100, K = 4, _lambda = _lambda, diff_coef = 2)
        varying_lambda_result_set.append([first_result, second_result])
        
    print(">> Job done!")
    return varying_steps_result_set, varying_K_result_set, varying_lambda_result_set

def plot_result(data, varying_variable, variable_name, image_name, image_type, details):
    rows, cols = 2, 3
    fig, ax = plt.subplots(rows, cols,
                           sharex='col', 
                           sharey='row')

    j = 0
    for row in range(2):
        i = 0
        for col in range(3):
            ax[row, col].imshow(data[i][j], cmap='gray', aspect='auto')
            i += 1
        j += 1

    ax[0, 0].set(ylabel='1ra variante de C(.)')
    ax[1, 0].set(xlabel='{} = {}'.format(variable_name, varying_variable[0]), ylabel='2da variante de C(.)')
    ax[1, 1].set(xlabel='{} = {}'.format(variable_name, varying_variable[1]))
    ax[1, 2].set(xlabel='{} = {}'.format(variable_name, varying_variable[2]))
    plt.suptitle("Imagen '{}.png' {} - {}".format(image_name, image_type, details))
    plt.savefig("output/varying-{}_{}png_{}.png".format(variable_name, image_name, image_type))
    plt.clf()

image_set = load_images()
steps_list = [100, 200, 300]
K_list = [2, 4, 8]
lambda_list = [0.083, 0.25, 1]

i = 0
while i < len(image_set):
    print("---------------------------------")
    print(">> -- Working with image {}.png --".format(str(i//2)))
    steps1, K1, lambda1 = perform_anisotropic_diffusion(image_set[i], steps_list, K_list, lambda_list)
    steps2, K2, lambda2 = perform_anisotropic_diffusion(image_set[i+1], steps_list, K_list, lambda_list)
    
    plot_result(steps1, steps_list, 'steps', str(i//2), 'original', 'K = 4 and lambda = 0.25')
    plot_result(steps2, steps_list, 'steps', str(i//2), 'noisy', 'K = 4 and lambda = 0.25')
    
    plot_result(K1, K_list, 'K', str(i//2), 'original', 'steps = 100 and lambda = 0.25')
    plot_result(K2, K_list, 'K', str(i//2), 'noisy', 'steps = 100 and lambda = 0.25')
    
    plot_result(lambda1, lambda_list, 'lambda', str(i//2), 'original', 'steps = 100 and K = 4')
    plot_result(lambda2, lambda_list, 'lambda', str(i//2), 'noisy', 'steps = 100 and K = 4')
    
    i += 2
    
print("---------------------------------")
print('>> Program ended <<')

----------------------------------
>> -- Working with image 0.png --
>> Anisotropic Diffusion evaluation has started!
>> Job done!
>> Anisotropic Diffusion evaluation has started!
>> Job done!
----------------------------------
>> -- Working with image 1.png --
>> Anisotropic Diffusion evaluation has started!
>> Job done!
>> Anisotropic Diffusion evaluation has started!
>> Job done!
----------------------------------
>> -- Working with image 2.png --
>> Anisotropic Diffusion evaluation has started!
>> Job done!
>> Anisotropic Diffusion evaluation has started!
>> Job done!
----------------------------------
>> -- Working with image 3.png --
>> Anisotropic Diffusion evaluation has started!
>> Job done!
>> Anisotropic Diffusion evaluation has started!
>> Job done!
----------------------------------
>> -- Working with image 4.png --
>> Anisotropic Diffusion evaluation has started!
>> Job done!
>> Anisotropic Diffusion evaluation has started!
>> Job done!
>> Program ended <<
